In [ ]:
import speech_recognition as sr
import time
import datetime
import os
import random
import ollama
import json
import glob
from TTS.api import TTS
import simpleaudio as sa
from mutagen.mp3 import MP3
from mutagen.easyid3 import EasyID3
import pygame
import numpy as np
from difflib import SequenceMatcher
import re
import requests

# ----------------------------
# Config
# ----------------------------
MODEL_NAME = "gemma2:9b"
VOICE_MODEL = "tts_models/en/ljspeech/tacotron2-DDC"
MUSIC_FOLDER = r"C:\Users\DIYOR\Music"  # Change to your music folder
URL = "http://172.20.10.2:80/toggle"
response=""

recognizer = sr.Recognizer()
mic = sr.Microphone()

pygame.mixer.init()

# Track current playing song
current_song_index = -1
music_files_list = []
current_playing_song = None

# ----------------------------
# Initialize TTS
# ----------------------------
print("🔊 Loading TTS voice model...")
tts = TTS(model_name=VOICE_MODEL, progress_bar=False, gpu=True)
print("✅ Voice model loaded.")

# ----------------------------
# Enhanced Command Matching
# ----------------------------
class EnhancedCommandRecognizer:
    def __init__(self, commands_file="alex_commands.json"):
        self.commands = self.load_commands(commands_file)
        self.training_matrix = {}
        self.context_history = []
        self.accuracy_threshold = 0.65
        self.confidence_levels = {"high": 0.85, "medium": 0.70, "low": 0.60}
        self.initialize_training_matrix()
        self.wake_word = "alex"
        
    def load_commands(self, filename):
        try:
            with open(filename, "r") as f:
                return json.load(f)
        except FileNotFoundError:
            print(f"⚠️  {filename} not found. Using empty commands.")
            return {}
    
    def initialize_training_matrix(self):
        for cmd_name, cmd_data in self.commands.items():
            keywords = cmd_data.get("keywords", [])
            for keyword in keywords:
                pattern = self.preprocess_text(keyword)
                self.training_matrix[pattern] = {
                    "command": cmd_name,
                    "action": cmd_data.get("action", ""),
                    "response": cmd_data.get("response", ""),
                    "response_function": cmd_data.get("response_function", ""),
                    "count": 0,
                    "accuracy_scores": [],
                    "last_used": None,
                    "success_rate": 1.0,
                    "failed_count": 0,
                    "avg_response_time": 0,
                    "first_used": None
                }
    
    def preprocess_text(self, text):
        text = text.lower().strip()
        text = re.sub(r'[^\w\s]', '', text)
        text = re.sub(r'\s+', ' ', text)
        return text
    
    def extract_command(self, user_input):
        user_input = self.preprocess_text(user_input)
        if user_input.startswith(self.wake_word):
            return user_input[len(self.wake_word):].strip(), True
        elif f" {self.wake_word} " in user_input:
            parts = user_input.split(self.wake_word, 1)
            return parts[1].strip() if len(parts) > 1 else user_input, True
        else:
            return user_input, False
    
    def calculate_similarity(self, user_input, pattern):
        user_input = self.preprocess_text(user_input)
        pattern = self.preprocess_text(pattern)
        if not user_input or not pattern:
            return 0
        base_similarity = SequenceMatcher(None, user_input, pattern).ratio()
        user_words = set(user_input.split())
        pattern_words = set(pattern.split())
        word_overlap = len(user_words.intersection(pattern_words)) / len(pattern_words) if pattern_words else 0
        overlap_bonus = word_overlap * 0.2
        if "off" in user_input and "off" in pattern:
            base_similarity = max(base_similarity, 0.95)
        if "on" in user_input and "on" in pattern:
            base_similarity = max(base_similarity, 0.95)
        if ("off" in user_input and "on" in pattern) or ("on" in user_input and "off" in pattern):
            base_similarity = max(base_similarity - 0.3, 0)
        if pattern in user_input:
            base_similarity = max(base_similarity, 0.9)
        return min(base_similarity + overlap_bonus, 1.0)
    
    def find_best_match(self, user_input, start_time=None):
        command_only, has_wake_word = self.extract_command(user_input)
        if not has_wake_word:
            return None, 0
        user_input = command_only
        best_match = None
        best_score = 0
        matched_pattern = ""
        for pattern, cmd_info in self.training_matrix.items():
            if not pattern:
                continue
            similarity = self.calculate_similarity(user_input, pattern)
            if similarity > best_score:
                best_score = similarity
                best_match = cmd_info.copy()
                matched_pattern = pattern
        if best_match and best_score >= self.accuracy_threshold and matched_pattern in self.training_matrix:
            self.training_matrix[matched_pattern]["count"] += 1
            self.training_matrix[matched_pattern]["accuracy_scores"].append(best_score)
            now = time.time()
            self.training_matrix[matched_pattern]["last_used"] = now
            if not self.training_matrix[matched_pattern]["first_used"]:
                self.training_matrix[matched_pattern]["first_used"] = now
            if start_time:
                response_time = time.time() - start_time
                count = self.training_matrix[matched_pattern]["count"]
                current_avg = self.training_matrix[matched_pattern]["avg_response_time"]
                self.training_matrix[matched_pattern]["avg_response_time"] = (current_avg * (count - 1) + response_time) / count
            if len(self.training_matrix[matched_pattern]["accuracy_scores"]) > 20:
                self.training_matrix[matched_pattern]["accuracy_scores"].pop(0)
            self.context_history.append({
                "input": user_input,
                "pattern": matched_pattern,
                "score": best_score,
                "timestamp": now
            })
            if len(self.context_history) > 10:
                self.context_history.pop(0)
        return best_match, best_score
    
    def get_confidence_level(self, score):
        if score >= self.confidence_levels["high"]:
            return "high", "💯"
        elif score >= self.confidence_levels["medium"]:
            return "medium", "✅"
        elif score >= self.confidence_levels["low"]:
            return "low", "⚠️"
        else:
            return "very low", "❌"

command_recognizer = EnhancedCommandRecognizer()

# ----------------------------
# Helper Functions
# ----------------------------
def speak(text):
    try:
        print("🤖 Alex says:", text)
        file_name = "alex_speak.wav"
        tts.tts_to_file(text=text, file_path=file_name)
        wave_obj = sa.WaveObject.from_wave_file(file_name)
        play_obj = wave_obj.play()
        play_obj.wait_done()
        time.sleep(0.1)
    except Exception as e:
        print("TTS error:", e)

def listen_with_noise_filter(seconds=5):
    with mic as source:
        try:
            recognizer.adjust_for_ambient_noise(source, duration=0.3)
            audio = recognizer.listen(source, timeout=seconds, phrase_time_limit=seconds)
            text = recognizer.recognize_google(audio, language="en-US").lower()
            if len(text.strip()) < 2:
                return ""
            if "alex" in text:
                print("🗣 You said:", text)
                return text
            else:
                return ""
        except sr.WaitTimeoutError:
            return ""
        except sr.UnknownValueError:
            return ""
        except Exception as e:
            print(f"Listening error: {e}")
            return ""

def ask_gpt(prompt):
    try:
        response = ollama.chat(
            model=MODEL_NAME,
            messages=[
                {"role": "system", "content": "You are a helpful AI voice assistant named Alex. Keep responses concise."},
                {"role": "user", "content": prompt}
            ]
        )
        return response['message']['content'].strip()
    except Exception as e:
        print("Gemma2 error:", e)
        return ""

def tell_joke():
    jokes = [
        "Why did the computer catch a cold? Because it had a bad Windows update!",
        "Why do programmers prefer dark mode? Because light attracts bugs!",
        "What's a computer's favorite snack? Microchips!"
    ]
    return random.choice(jokes)

def get_time():
    now = datetime.datetime.now()
    return now.strftime("It's %I:%M %p.")

def get_date():
    today = datetime.date.today()
    return today.strftime("Today is %B %d, %Y.")

device_states = {"light": False, "music": False}

def toggle_light(state=None):
    if state is True:
        device_states["light"] = True
        return "Turning on the light. Bright and shiny!"
    elif state is False:
        device_states["light"] = False
        return "Turning off the light. Good night!"
    else:
        device_states["light"] = not device_states["light"]
        return "Turning on the light. Bright and shiny!" if device_states["light"] else "Turning off the light. Good night!"

def export_training_data():
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"alex_training_data_{timestamp}.json"
    try:
        stats = {
            "training_matrix": command_recognizer.training_matrix,
            "context_history": command_recognizer.context_history,
            "export_time": timestamp,
            "device_states": device_states
        }
        with open(filename, "w") as f:
            json.dump(stats, f, indent=2, default=str)
        return f"Training data exported to {filename}"
    except Exception as e:
        print(f"Export error: {e}")
        return "Failed to export training data"

# ----------------------------
# Music Functions
# ----------------------------
def load_music_files():
    global music_files_list
    music_files_list = glob.glob(os.path.join(MUSIC_FOLDER, "*.mp3"))
    return music_files_list

def play_music(command=None):
    global current_song_index, music_files_list, current_playing_song
    if not music_files_list:
        music_files_list = load_music_files()
    if not music_files_list:
        speak("No music found in your Music folder.")
        device_states["music"] = False
        return

    selected_song = None
    command_artist = None
    if command and "by " in command.lower():
        command_artist = command.lower().split("by ")[-1].strip()

    if command_artist:
        artist_songs = []
        for song_path in music_files_list:
            try:
                artist = MP3(song_path, ID3=EasyID3).get("artist", [""])[0].lower()
            except:
                artist = ""
            if command_artist in artist:
                artist_songs.append(song_path)
        if artist_songs:
            selected_song = random.choice(artist_songs)
            current_song_index = music_files_list.index(selected_song)

    if not selected_song:
        current_song_index = (current_song_index + 1) % len(music_files_list) if 0 <= current_song_index < len(music_files_list) else random.randint(0, len(music_files_list) - 1)
        selected_song = music_files_list[current_song_index]

    current_playing_song = selected_song
    song_name = os.path.basename(selected_song).replace(".mp3", "")
    try:
        audio_meta = MP3(selected_song, ID3=EasyID3)
        title = audio_meta.get("title", [song_name])[0]
        artist = audio_meta.get("artist", ["Unknown Artist"])[0]
        speak(f"Playing {title} by {artist}")
    except:
        speak(f"Playing {song_name}")

    pygame.mixer.music.load(selected_song)
    pygame.mixer.music.play()
    device_states["music"] = True

def stop_music():
    if pygame.mixer.music.get_busy():
        pygame.mixer.music.stop()
        device_states["music"] = False

def next_music():
    global current_song_index, music_files_list, current_playing_song
    if not device_states["music"] or not music_files_list:
        speak("No music is currently playing.")
        return
    if pygame.mixer.music.get_busy():
        pygame.mixer.music.stop()
    current_song_index = (current_song_index + 1) % len(music_files_list)
    selected_song = music_files_list[current_song_index]
    current_playing_song = selected_song
    song_name = os.path.basename(selected_song).replace(".mp3", "")
    try:
        audio_meta = MP3(selected_song, ID3=EasyID3)
        title = audio_meta.get("title", [song_name])[0]
        artist = audio_meta.get("artist", ["Unknown Artist"])[0]
        speak(f"Next song: {title} by {artist}")
    except:
        speak(f"Next song: {song_name}")
    pygame.mixer.music.load(selected_song)
    pygame.mixer.music.play()

# ----------------------------
# Command Handler
# ----------------------------
def handle_command(user_input):
    start_time = time.time()
    best_match, accuracy_score = command_recognizer.find_best_match(user_input, start_time)
    conf_level, emoji = command_recognizer.get_confidence_level(accuracy_score)
    print(f"{emoji} Command matching: {accuracy_score:.1%} ({conf_level} confidence)")

    if "export training data" in user_input.lower():
        speak(export_training_data())
        return "handled"

    if best_match and accuracy_score >= command_recognizer.accuracy_threshold:
        command_name = best_match.get("command", "")
        action = best_match.get("action", "")
        response = best_match.get("response", "")
        response_function = best_match.get("response_function", "")
        try:
            # 🆕 Print command when saying "Alex turn on light"
            if command_name == "light_on":
                
                cmd = "ON"
                response = requests.post(URL, data=cmd)
                print(f"🗣 Recognized Command: '{user_input}'")
                speak(toggle_light(True))
                return "handled"

            elif command_name == "light_off":
                cmd = "OFF"
                response = requests.post(URL, data=cmd)
                print(f"🗣 Recognized Command: '{user_input}'")
                
                speak(toggle_light(False))
                return "handled"

            elif action == "music":
                play_music(user_input)
                return "handled"

            elif action == "stop_music":
                stop_music()
                if response:
                    speak(response)
                return "handled"

            elif action == "next_music":
                next_music()
                return "handled"

            elif action == "notepad":
                speak(response)
                os.system("notepad")
                return "handled"

            elif action == "sleep":
                active_patterns = [(p, info) for p, info in command_recognizer.training_matrix.items() if info['count'] > 0]
                print("\n" + "=" * 60)
                print("📋 ALEX TRAINING STATISTICS")
                print("=" * 60)
                print(f"Total Interactions: {sum(info['count'] for info in command_recognizer.training_matrix.values())}")
                print(f"Active Commands: {len(active_patterns)}")
                if active_patterns:
                    print("\n📊 Command Performance:")
                    for pattern, info in active_patterns:
                        if info['accuracy_scores']:
                            avg_acc = np.mean(info['accuracy_scores'])
                            print(f"  • {pattern}: {info['count']} uses, {avg_acc:.1%} accuracy")
                print(f"\n💡 Device States:")
                for device, state in device_states.items():
                    print(f"  • {device}: {'ON' if state else 'OFF'}")
                if current_playing_song:
                    song_name = os.path.basename(current_playing_song).replace(".mp3", "")
                    print(f"\n🎵 Current Song: {song_name}")
                print("=" * 60)
                speak("Showing training statistics. Going to sleep now.")
                return "sleep_with_stats"

            elif response_function:
                func = globals().get(response_function)
                if func:
                    speak(func())
                return "handled"

            elif response:
                speak(response)
                return "handled"
        except Exception as e:
            print(f"Command execution error: {e}")
            speak("Sorry, No internet connection.")
            return "error"

    if accuracy_score > 0.3 and len(user_input.split()) > 2:
        print(f"❌ No match found (score: {accuracy_score:.1%})")
        gpt_reply = ask_gpt(user_input)
        if gpt_reply:
            speak(gpt_reply)
        return "not_handled"
    else:
        return "ignored"

# ----------------------------
# Main Loop
# ----------------------------
print("🎤 Alex is listening... Say 'Alex' followed by your command.")
print("💡 Examples: 'Alex play music', 'Alex next music', 'Alex skip song'")
print("🔇 Will only respond to commands starting with 'Alex'")
print("💤 Say 'Alex go to sleep' to see training statistics")

interaction_count = 0
last_command_time = 0
COMMAND_COOLDOWN = 1.5

load_music_files()
if music_files_list:
    print(f"🎵 Loaded {len(music_files_list)} music files")

while True:
    try:
        user_input = listen_with_noise_filter(seconds=5)
        current_time = time.time()
        if current_time - last_command_time < COMMAND_COOLDOWN:
            continue
        if not user_input:
            continue
        last_command_time = current_time
        interaction_count += 1
        result = handle_command(user_input)
        if interaction_count % 5 == 0:
            print(f"\n📊 Update: {interaction_count} interactions completed")
        if result == "sleep_with_stats":
            print("\n💤 Alex is now sleeping...")
            sleep_start = time.time()
            while time.time() - sleep_start < 30:
                sleep_input = listen_with_noise_filter(seconds=3)
                if sleep_input:
                    print("🔔 Waking up from sleep...")
                    break
                time.sleep(0.5)
            print("🎤 Alex is listening again...")
    except KeyboardInterrupt:
        print("\n👋 Shutting down Alex...")
        print("\n📋 Final device states:")
        for device, state in device_states.items():
            print(f"  {device}: {'ON' if state else 'OFF'}")
        if current_playing_song:
            song_name = os.path.basename(current_playing_song).replace(".mp3", "")
            print(f"🎵 Last played song: {song_name}")
        break
    except Exception as e:
        print(f"Main loop error: {e}")
        continue
